In [1]:
  pip install datasets


Note: you may need to restart the kernel to use updated packages.


In [2]:
  from datasets import load_dataset

  dataset = load_dataset('cnn_dailymail', '3.0.0')


In [3]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})


In [4]:
import pandas as pd
df=pd.DataFrame(dataset["train"])

In [5]:
df.head()

,article,highlights,id
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",06352019a19ae31e527f37f7571c6dd7f0c5da37
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a


In [6]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287113 entries, 0 to 287112
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   article     287113 non-null  object
 1   highlights  287113 non-null  object
 2   id          287113 non-null  object
dtypes: object(3)
memory usage: 6.6+ MB
None


In [8]:
print(df.isnull().sum())

article       0
highlights    0
id            0
dtype: int64


In [7]:
pip install transformers datasets


Note: you may need to restart the kernel to use updated packages.


In [8]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
print("loaded")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


loaded


In [10]:
def tokenize_function(example):
    input_texts = ["summarize: " + text for text in example["article"]]  
    target_texts = [text for text in example["highlights"]]  

    inputs = tokenizer(input_texts, padding="max_length", truncation=True, max_length=512)
    targets = tokenizer(target_texts, padding="max_length", truncation=True, max_length=128)

    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": targets["input_ids"]  
    }


In [11]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [12]:
import pandas as pd


df_train = pd.DataFrame(tokenized_dataset["train"])
df_val = pd.DataFrame(tokenized_dataset["validation"])
df_test = pd.DataFrame(tokenized_dataset["test"])


df_train.to_parquet("/kaggle/working/tokenized_train.parquet")
df_val.to_parquet("/kaggle/working/tokenized_val.parquet")
df_test.to_parquet("/kaggle/working/tokenized_test.parquet")

print("Tokenized dataset saved successfully!")


✅ Tokenized dataset saved successfully!


In [15]:
!zip -r /kaggle/working/tokenized_data.zip /kaggle/working/tokenized_*.parquet


  adding: kaggle/working/tokenized_test.parquet (deflated 14%)
  adding: kaggle/working/tokenized_train.parquet (deflated 13%)
  adding: kaggle/working/tokenized_val.parquet (deflated 14%)


In [20]:
!mkdir -p /kaggle/working/my_dataset
!ls -l /kaggle/working/


total 2121852
drwxr-xr-x 2 root root       4096 Mar 26 15:21 my_dataset
-rw-r--r-- 1 root root 1007975482 Mar 26 15:09 tokenized_data.zip
-rw-r--r-- 1 root root   42295413 Mar 26 14:59 tokenized_test.parquet
-rw-r--r-- 1 root root 1073435272 Mar 26 14:58 tokenized_train.parquet
-rw-r--r-- 1 root root   49045550 Mar 26 14:58 tokenized_val.parquet


In [21]:
!mv /kaggle/working/tokenized_data.zip /kaggle/working/my_dataset/


In [26]:
!mkdir -p /root/.kaggle  
!cp /kaggle/input/json123/kaggle.json /root/.kaggle/ 
!chmod 600 /root/.kaggle/kaggle.json s


In [27]:
!kaggle datasets list


ref                                                               title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
atharvasoundankar/chocolate-sales                                 Chocolate Sales Data 📊🍫                              14KB  2025-03-19 03:51:40          11944        210  1.0              
adilshamim8/student-depression-dataset                            Student Depression Dataset                          456KB  2025-03-13 03:12:30           4234         68  1.0              
abdulmalik1518/mobiles-dataset-2025                               Mobiles Dataset (2025)                               20KB  2025-02-18 06:50:24          17560        293  1.0              
ak0212/average-daily-screen-time-for-children     

In [33]:
%%writefile /kaggle/working/my_dataset/dataset-metadata.json
{
  "title": "Tokenized Dataset",
  "id": "ayaanakhter/tokenized-dataset",
  "licenses": [{"name": "CC0-1.0"}]
}


Overwriting /kaggle/working/my_dataset/dataset-metadata.json


In [34]:
!kaggle datasets create -p /kaggle/working/my_dataset


Starting upload for file tokenized_data.zip
100%|████████████████████████████████████████| 961M/961M [00:23<00:00, 42.9MB/s]
Upload successful: tokenized_data.zip (961MB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/ayaanakhter/tokenized-dataset


In [23]:
!kaggle datasets create -p /kaggle/working/my_dataset -m "Saving tokenized dataset"


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [9]:
import pandas as pd


df_train = pd.read_parquet("/kaggle/input/tokenized-dataset/kaggle/working/tokenized_test.parquet")
df_val = pd.read_parquet("/kaggle/input/tokenized-dataset/kaggle/working/tokenized_train.parquet")
df_test = pd.read_parquet("/kaggle/input/tokenized-dataset/kaggle/working/tokenized_val.parquet")

print("Loaded Successfully!")


Loaded Successfully!


In [13]:
from datasets import load_dataset


dataset = load_dataset("parquet", data_files={
    "train": "/kaggle/input/tokenized-dataset/kaggle/working/tokenized_test.parquet",
    "validation": "/kaggle/input/tokenized-dataset/kaggle/working/tokenized_train.parquet",
    "test": "/kaggle/input/tokenized-dataset/kaggle/working/tokenized_val.parquet"
}, streaming=True)

print(" Tokenized Dataset Loaded Using Streaming!")


 Tokenized Dataset Loaded Using Streaming!


In [14]:

for sample in dataset["train"].take(1):  
    print(sample)


{'article': '(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday

In [15]:
from transformers import T5ForConditionalGeneration, T5Tokenizer


model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

print(" Flan-T5 Model Loaded!")


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

 Flan-T5 Model Loaded!


In [16]:
import torch


from torch.utils.data import DataLoader

def collate_fn(batch):
    input_ids = torch.tensor([sample["input_ids"] for sample in batch])
    attention_mask = torch.tensor([sample["attention_mask"] for sample in batch])
    labels = torch.tensor([sample["labels"] for sample in batch])
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

train_dataloader = DataLoader(dataset["train"], batch_size=2, collate_fn=collate_fn)
val_dataloader = DataLoader(dataset["validation"], batch_size=2, collate_fn=collate_fn)

print("✅ DataLoader is Ready!")


✅ DataLoader is Ready!


In [20]:
import time
from datasets import load_dataset, IterableDatasetDict
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
import torch


start_time = time.time()


dataset = IterableDatasetDict(dataset)


model_checkpoint = "/kaggle/working/fine_tuned_flan_t5_checkpoint"
try:
    model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)  
    tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)
    print("✅ Resuming from last saved checkpoint!")
except:
    model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
    tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
    print("✅ Loaded Flan-T5 base model!")


training_args = TrainingArguments(
    output_dir="/kaggle/working/flan_t5_summarization",
    evaluation_strategy="epoch",
    save_strategy="epoch",  
    per_device_train_batch_size=2,  
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    fp16=True,  
    save_total_limit=2,  
    logging_steps=100,
    report_to="none",
    load_best_model_at_end=True,  
    max_steps=10000  
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"]
)


train_start_time = time.time()
trainer.train()
train_end_time = time.time()


model.save_pretrained("/kaggle/working/fine_tuned_flan_t5")
tokenizer.save_pretrained("/kaggle/working/fine_tuned_flan_t5")

e
total_time = (train_end_time - train_start_time) / 60  # Convert to minutes
end_time = time.time()
total_script_time = (end_time - start_time) / 60  # Total script execution time

print(f"✅ Training complete! Model saved. 🕒 Training Time: {total_time:.2f} minutes")
print(f"🚀 Total script execution time: {total_script_time:.2f} minutes")


✅ Loaded Flan-T5 base model!


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
0,0.000000,nan
1,0.000000,nan


/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3161: RuntimeWarning: invalid value encountered in less
  if operator(metric_value, self.state.best_metric):
/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3161: RuntimeWarning: invalid value encountered in less
  if operator(metric_value, self.state.best_metric):


✅ Training complete! Model saved. 🕒 Training Time: 45.29 minutes
🚀 Total script execution time: 45.36 minutes


In [9]:
!ls -lh /kaggle/working/flan_t5_summarization/checkpoint-10000

total 945M
-rw-r--r-- 1 root root 1.6K Mar 26 17:23 config.json
-rw-r--r-- 1 root root  142 Mar 26 17:23 generation_config.json
-rw-r--r-- 1 root root 945M Mar 26 17:23 model.safetensors
-rw-r--r-- 1 root root 1.8K Mar 26 17:23 optimizer.pt
-rw-r--r-- 1 root root  14K Mar 26 17:23 rng_state.pth
-rw-r--r-- 1 root root 1.1K Mar 26 17:23 scheduler.pt
-rw-r--r-- 1 root root  14K Mar 26 17:23 trainer_state.json
-rw-r--r-- 1 root root 5.3K Mar 26 17:23 training_args.bin


In [11]:
!ls -lh /kaggle/working/flan_t5_summarization/checkpoint-10000


total 945M
-rw-r--r-- 1 root root 1.6K Mar 26 17:23 config.json
-rw-r--r-- 1 root root  142 Mar 26 17:23 generation_config.json
-rw-r--r-- 1 root root 945M Mar 26 17:23 model.safetensors
-rw-r--r-- 1 root root 1.8K Mar 26 17:23 optimizer.pt
-rw-r--r-- 1 root root  14K Mar 26 17:23 rng_state.pth
-rw-r--r-- 1 root root 1.1K Mar 26 17:23 scheduler.pt
-rw-r--r-- 1 root root  14K Mar 26 17:23 trainer_state.json
-rw-r--r-- 1 root root 5.3K Mar 26 17:23 training_args.bin


In [12]:
from transformers import T5ForConditionalGeneration, T5Tokenizer


checkpoint_path = "/kaggle/working/flan_t5_summarization/checkpoint-10000"
model = T5ForConditionalGeneration.from_pretrained(checkpoint_path, use_safetensors=True)


tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

print("✅ Model reloaded successfully from checkpoint-10000!")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


✅ Model reloaded successfully from checkpoint-10000!


In [13]:

model.save_pretrained("/kaggle/working/fine_tuned_flan_t5")
tokenizer.save_pretrained("/kaggle/working/fine_tuned_flan_t5")


!zip -r /kaggle/working/fine_tuned_flan_t5.zip /kaggle/working/fine_tuned_flan_t5/


  adding: kaggle/working/fine_tuned_flan_t5/ (stored 0%)
  adding: kaggle/working/fine_tuned_flan_t5/special_tokens_map.json (deflated 85%)
  adding: kaggle/working/fine_tuned_flan_t5/generation_config.json (deflated 29%)
  adding: kaggle/working/fine_tuned_flan_t5/added_tokens.json (deflated 83%)
  adding: kaggle/working/fine_tuned_flan_t5/tokenizer_config.json (deflated 94%)
  adding: kaggle/working/fine_tuned_flan_t5/spiece.model (deflated 48%)
  adding: kaggle/working/fine_tuned_flan_t5/model.safetensors (deflated 7%)
  adding: kaggle/working/fine_tuned_flan_t5/config.json (deflated 61%)


In [14]:
!kaggle datasets init -p /kaggle/working/fine_tuned_flan_t5/


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [15]:
import shutil
import os


os.makedirs("/root/.kaggle", exist_ok=True)


shutil.copy("/kaggle/input/json123/kaggle.json", "/root/.kaggle/kaggle.json")


os.chmod("/root/.kaggle/kaggle.json", 600)

print("Kaggle API key set up successfully!")


Kaggle API key set up successfully!


In [16]:
!kaggle datasets list


ref                                                               title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
atharvasoundankar/chocolate-sales                                 Chocolate Sales Data 📊🍫                              14KB  2025-03-19 03:51:40          12038        211  1.0              
adilshamim8/student-depression-dataset                            Student Depression Dataset                          456KB  2025-03-13 03:12:30           4317         68  1.0              
abdulmalik1518/mobiles-dataset-2025                               Mobiles Dataset (2025)                               20KB  2025-02-18 06:50:24          17620        294  1.0              
ak0212/average-daily-screen-time-for-children     

In [18]:
!kaggle datasets init -p /kaggle/working/fine_tuned_flan_t5/


Data package template written to: /kaggle/working/fine_tuned_flan_t5/dataset-metadata.json


In [22]:
!cat /kaggle/working/fine_tuned_flan_t5/dataset-metadata.json


{
  "title": "INSERT_TITLE_HERE",
  "id": "ayaanakhter/INSERT_SLUG_HERE",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}

In [23]:
%%writefile /kaggle/working/fine_tuned_flan_t5/dataset-metadata.json
{
  "title": "Fine-Tuned Flan-T5",
  "id": "ayaanakhter/fine-tuned-flan-t5",  
  "licenses": [{"name": "CC0-1.0"}]
}


Overwriting /kaggle/working/fine_tuned_flan_t5/dataset-metadata.json


In [24]:
!kaggle datasets create -p /kaggle/working/fine_tuned_flan_t5/


Starting upload for file special_tokens_map.json
100%|██████████████████████████████████████| 2.48k/2.48k [00:00<00:00, 11.3kB/s]
Upload successful: special_tokens_map.json (2KB)
Starting upload for file generation_config.json
100%|████████████████████████████████████████████| 142/142 [00:00<00:00, 644B/s]
Upload successful: generation_config.json (142B)
Starting upload for file added_tokens.json
100%|██████████████████████████████████████| 2.53k/2.53k [00:00<00:00, 11.1kB/s]
Upload successful: added_tokens.json (3KB)
Starting upload for file tokenizer_config.json
100%|███████████████████████████████████████| 20.4k/20.4k [00:00<00:00, 113kB/s]
Upload successful: tokenizer_config.json (20KB)
Starting upload for file spiece.model
100%|████████████████████████████████████████| 773k/773k [00:00<00:00, 3.79MB/s]
Upload successful: spiece.model (773KB)
Starting upload for file model.safetensors
100%|█████████████████████████████████████████| 944M/944M [00:04<00:00, 199MB/s]
Upload successful

In [41]:
sample_article = """LONDON, England (CNN) -- Previously unseen footage of Diana, Princess of Wales, taken just hours 
before she was killed in a car crash, has been shown to the jury at the inquest into her death. The footage showed Diana 
and Dodi step into an elevator at the Ritz Hotel. Images taken from a security camera at the Ritz Hotel in Paris show the 
36-year-old smiling as she and her lover Dodi Fayed step into an elevator and later walk out of the hotel.

Further footage shows Fayed visiting a jeweler's shop, images that could lend support to claims that he was buying an 
engagement ring. Earlier, a British coroner at the inquest said it may never be known for certain whether Princess Diana 
was pregnant when she died in the Paris car crash. Lord Justice Scott Baker told the jury at the inquest into the deaths 
of the princess and her lover Dodi Fayed that scientific evidence might be unable to demonstrate "one way or the other" 
whether she was in the early stages of pregnancy. 

But he said they would hear "intimate" details of her personal life. Watch footage of Diana's last hours. Baker told 
the 11 members of the jury -- six women and five men -- Diana may have been on the contraceptive pill and that evidence 
she was poised to get engaged to Dodi on the night she died was contradictory.
"""

In [42]:
from transformers import T5ForConditionalGeneration, T5Tokenizer


model_path = "/kaggle/input/fine-tuned-flan-t5"
model = T5ForConditionalGeneration.from_pretrained(model_path, use_safetensors=True)
tokenizer = T5Tokenizer.from_pretrained(model_path)

print(" Fine-Tuned Flan-T5 Loaded ")


def summarize_text(text):
    try:
        inputs = tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
        summary_ids = model.generate(
            **inputs,
            max_length=256, 
            num_beams=5,  
            length_penalty=1.2,  
            no_repeat_ngram_size=3,  
            early_stopping=True
        )
        return tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    except Exception as e:
        return f"Error: {str(e)}"

generated_summary = summarize_text(sample_article)
print("\ Generated Summary:")
print(generated_summary)

 Fine-Tuned Flan-T5 Loaded 
\ Generated Summary:
Dodi Fayed, 36, died in a car crash at the Ritz Hotel in Paris.


In [40]:
sample_article = {
    "news": """The global economy is experiencing a slowdown due to multiple factors, including inflation, 
    supply chain disruptions, and geopolitical tensions. Central banks worldwide are increasing interest rates 
    to curb inflation, but this has led to reduced consumer spending and business investments. Many countries 
    are also facing energy crises, leading to higher costs for households and industries. Economists predict a 
    period of stagflation, where economic growth remains stagnant while inflation continues to rise."""
}

generated_summary = summarize_text(sample_article["news"])
print("Generated Summary:", generated_summary)


Generated Summary: The global economy is experiencing a slowdown due to a number of factors, including inflation and supply chain disruptions.


In [43]:
challenging_text = """Quantum mechanics, a fundamental theory in physics, describes the behavior of matter and energy on atomic and subatomic scales. Unlike classical mechanics, where objects follow deterministic paths, quantum mechanics introduces probabilistic outcomes dictated by wave functions. The Heisenberg Uncertainty Principle asserts that one cannot simultaneously determine both the position and momentum of a particle with absolute precision. Superposition, another core concept, posits that particles exist in multiple states until measured. Entanglement, described by Einstein as "spooky action at a distance," allows two particles to share a state regardless of distance, defying classical notions of locality.

Despite its abstract nature, quantum mechanics underpins modern technologies such as semiconductors, quantum computing, and cryptography. However, the transition from theoretical physics to practical applications presents challenges. Quantum computers, which leverage qubits instead of classical bits, promise exponential speed-ups for certain problems, yet they remain constrained by decoherence and error rates. The ongoing debate between the Copenhagen interpretation, which suggests wave function collapse upon measurement, and the Many-Worlds interpretation, which proposes parallel realities, underscores the philosophical depth of quantum mechanics.

Moreover, recent advancements in quantum field theory and quantum gravity seek to unify general relativity with quantum mechanics, a pursuit that has eluded physicists for decades. The incompatibility between Einstein’s smooth spacetime and quantum fluctuations suggests the need for a deeper framework, potentially involving string theory or loop quantum gravity. Experimental evidence from high-energy particle collisions and cosmological observations may provide clues, yet a complete theory of quantum gravity remains an open question, challenging our fundamental understanding of the universe itself."""

generated_summary = summarize_text(challenging_text)
print("Generated Summary:", generated_summary)

Generated Summary: Quantum mechanics describes the behavior of matter and energy on atomic and subatomic scales.
